In [127]:
#!/Tsan/bin/python
# -*- coding: utf-8 -*-

In [128]:
# Libraries To Use
from __future__ import division 
import numpy as np
import pandas as pd
import statsmodels.api as sm
import os
from sklearn import linear_model
from datetime import datetime,time,date
import matplotlib.pyplot as plt
import theano.tensor as T
from theano import function
import seaborn as sns
from theano.tensor.shared_randomstreams import RandomStreams

In [129]:
# Import My own library for factor testing
from SingleFactorTest import factorFilterFunctions as ff
#from config import *

In [130]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [131]:
path = ff.data_path # path

In [132]:
filenameFCAP = 'LZ_GPA_VAL_A_FCAP.csv'

In [133]:
# --------------------------------------- Global Functions to def---------------------------------- #

In [134]:
# top functions to def
def simpleNormalize(narrowedData):
    dataWinsorized = narrowedData.copy()
    dataWinsorizedTrans = dataWinsorized.T
    MAD = 1.483*np.abs(dataWinsorizedTrans-dataWinsorizedTrans.median(skipna=True))
    return ((dataWinsorizedTrans - dataWinsorizedTrans.mean(axis=0, skipna=True))/dataWinsorizedTrans.std(axis=0, skipna=True)).T

In [135]:
# top function
def neutralizeFactor(normalizedFactorDF, normalizedLFCAPDF, datelist):
    factorNeutralized = pd.DataFrame(index=normalizedFactorDF.index, columns=normalizedFactorDF.columns, data=None, dtype = float)
    for date in datelist:
        LFCAPIndice = normalizedLFCAPDF.loc[date].dropna()
        factorIndice = normalizedFactorDF.loc[date].dropna()
        intersectionStocks = list(set(LFCAPIndice.index) & set(factorIndice.index))
        #dummy_Matrix = pd.get_dummies(IndustryDF.loc[date]).T.iloc[:-1]
        #dummy_Matrix = dummy_Matrix[intersectionStocks].append(LFCAPIndice.loc[intersectionStocks])
        try:
            result = sm. OLS(factorIndice.loc[intersectionStocks].T, LFCAPIndice.loc[intersectionStocks].T).fit()
            factorNeutralized.loc[date][intersectionStocks] = result.resid
        except:
            factorNeutralized.loc[date] = np.NaN
    return factorNeutralized

In [136]:
# --------------------------------------- Function Section End ---------------------------------- #

In [137]:
# Data prepared for Neuralization
FCAP1 = np.log10(pd.read_csv(path+filenameFCAP,infer_datetime_format=True,parse_dates=[0],index_col=0))
NormalizedFCAP = simpleNormalize(FCAP1 )

In [138]:
# --------------------------------------- Calculate Forward Adjusted Price ---------------------------------- #

In [139]:
# the necessary files
filenameAdjustFactor='LZ_GPA_CMFTR_CUM_FACTOR.csv'
filenamePirce='LZ_GPA_QUOTE_TCLOSE.csv'

In [163]:
# first is to calculate forward adjusted pice
def calAdjustedPrice():
    # Adjusted factor
    AdjFacBackward=pd.read_csv(path+filenameAdjustFactor,infer_datetime_format=True,parse_dates=[0],index_col=0)
    #AdjFacBackward=AdjFacBackward[:]

    #PriceData to Adjust
    PriceToAdj=pd.read_csv(path+filenamePirce,infer_datetime_format=True,parse_dates=[0],index_col=0)
    #PriceToAdj=PriceToAdj[:]

    #Calculate
    AdjFacforward = AdjFacBackward / AdjFacBackward.max()
    adjustedPrice = (AdjFacforward*PriceToAdj).round(5)
    #adjustedPrice.index.name = 'LZ_GPA_DERI_AdjustedPriceForward_20-d' 
    adjustedPrice.index.name = 'Own_Factor_AdjustedPriceForward-1d'
    print adjustedPrice.index.name 
    adjustedPrice.to_csv(path+'Own_Factor_AdjustedPriceForward-1d.csv',na_rep='NaN',date_format='%Y%m%d')

In [164]:
calAdjustedPrice()

ValueError: operands could not be broadcast together with shapes (9009,) (3,) 

000005.SZ        9.393567
600601.SH     5974.691786
600602.SH      513.680061
600651.SH     5905.618552
600652.SH     5505.536186
600653.SH    10055.640083
600654.SH     1506.963701
600656.SH       17.770853
000004.SZ        4.063862
000002.SZ      133.886691
000001.SZ      107.166489
000009.SZ        7.347011
000003.SZ             NaN
600603.SH       77.283530
000012.SZ       17.618287
000016.SZ       20.695610
600604.SH       26.234874
600605.SH       35.010198
600606.SH      115.985090
600607.SH       78.096562
600608.SH       99.475227
000011.SZ        3.491401
000017.SZ        2.682738
000007.SZ        5.521192
000006.SZ       32.921011
000020.SZ        1.751825
000013.SZ             NaN
000008.SZ       22.050885
000014.SZ        5.427468
000018.SZ        6.632775
                 ...     
603096.SH        1.004437
002866.SZ        1.060259
300514.SZ        1.099885
603920.SH        1.004407
002867.SZ        1.061434
603787.SH        1.100132
603320.SH        1.100180
002868.SZ   

In [152]:
AdjFacBackward.max().shape

(3323L,)

In [ ]:
# --------------------------------------- Calculate ILLQ Factor(5-days average) ---------------------------------- #

In [ ]:
# 
filenameClose = 'LZ_GPA_QUOTE_TCLOSE.csv'
filenameOpen = 'LZ_GPA_QUOTE_TOPEN.csv'
filenameVolume = 'LZ_GPA_QUOTE_TVOLUME.csv'

In [ ]:
def calcILLQ(): # rolling_window is set as 5 days
    openPrice = pd.read_csv(path+filenameOpen,infer_datetime_format=True,parse_dates=[0],index_col=0)
    closePrice = pd.read_csv(path+filenameClose,infer_datetime_format=True,parse_dates=[0],index_col=0)
    volume = pd.read_csv(path+filenameVolume,infer_datetime_format=True,parse_dates=[0],index_col=0)
    if openPrice.shape != closePrice.shape:
        print openPrice.shape, closePrice.shape
        print 'data shape is not equal!'
    else:
        newdf = np.abs((closePrice - openPrice)/openPrice)/volume
        newdf = newdf.rolling(min_periods=5,window=5,center=False).mean()
        newdf.index.name = 'Own_Factor_ILLQ-1d'
        newdf.to_csv(path+'Own_Factor_ILLQ-1d.csv',na_rep='NaN',date_format='%Y%m%d')
        return newdf       

In [ ]:
#filenameILLQ = 'Own_Factor_ILLQ-1d.csv'
#openPrice = pd.read_csv(path+filenameILLQ,infer_datetime_format=True,parse_dates=[0],index_col=0)

In [ ]:
ILLQdf = calcILLQ()

In [ ]:
# TO nuetralize DDA20
NormalizedILLQ = simpleNormalize(ILLQdf)
neutralizedILLQ = neutralizeFactor(NormalizedILLQ, NormalizedFCAP, NormalizedILLQ.index)
neutralizedILLQ.index.name = 'Own_Factor_ADJ_ILLQ_1D'
neutralizedILLQ.to_csv(path+neutralizedILLQ.index.name+'.csv',na_rep='NaN',date_format='%Y%m%d')

In [ ]:
# --------------------------------------- Calculate FCAP Adjusted Turnover Volume---------------------------------- #

In [ ]:
# --------------------------------------- ILLIQ End--------------------------------- #

In [ ]:
# REMINDER: Code in this section can be used to neutralize any new factor! Use this to check some size-affected factor!!!

In [ ]:
filenameTURNOVER = 'LZ_GPA_QUOTE_TVOLUME.csv'
filenameFCAP = 'LZ_GPA_VAL_A_FCAP.csv'

In [ ]:
TURNOVER = pd.read_csv(path+filenameTURNOVER,infer_datetime_format=True,parse_dates=[0],index_col=0)
FCAP1 = np.log10(pd.read_csv(path+filenameFCAP,infer_datetime_format=True,parse_dates=[0],index_col=0))

TURNOVER= simpleNormalize(TURNOVER)

FCAP1 = simpleNormalize(FCAP1 )

datelist = FCAP1.index.tolist()

In [ ]:
adjustedTurnOver = neutralizeFactor(TURNOVER , FCAP1, datelist)

In [ ]:
adjustedTurnOver.index.name = 'Own_Factor_AdjustedTurnOver-1d'
adjustedTurnOver.to_csv(path+'Own_Factor_AdjustedTurnOver-1d.csv',na_rep='NaN',date_format='%Y%m%d')

In [ ]:
# --------------------------------------- Calculate FCAP Adjusted PB ---------------------------------- #

In [ ]:
filenamePB = 'LZ_GPA_VAL_PB.csv'
filenameFCAP = 'LZ_GPA_VAL_A_FCAP.csv'
PB = pd.read_csv(path+filenamePB ,infer_datetime_format=True,parse_dates=[0],index_col=0)
PB = simpleNormalize(PB)

In [ ]:
adjustedPB = neutralizeFactor(PB , FCAP1, datelist)
adjustedPB.index.name = 'Own_Factor_AdjustedPB-1d'
adjustedPB.to_csv(path+'Own_Factor_AdjustedPB-1d.csv',na_rep='NaN',date_format='%Y%m%d')

In [ ]:
pb1 = pd.read_csv(path+'Own_Factor_AdjustedPB-1d.csv',infer_datetime_format=True,parse_dates=[0],index_col=0)

In [ ]:
pb1.columns

In [ ]:
# --------------------------------------- Calculate  x-days return volatility  ---------------------------------- #

In [ ]:
filenameAdjPrice =  'Own_Factor_AdjustedPriceForward-1d.csv'

In [ ]:
def calReturnVol(period): # 90days maybe good
    AdjPrice = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0)
    returnDF = AdjPrice.pct_change()
    newdf = returnDF.rolling(min_periods=20,window=period,center=False).std()
    newdf.index.name = 'Own_Factor_Volatility_%dd' % period
    newdf.to_csv(path+'Own_Factor_Volatility_%dd.csv' % period,na_rep='NaN',date_format='%Y%m%d')
    return newdf

In [ ]:
calReturnVol(90).tail()

In [ ]:
# --------------------------------------- Calculate  x-days return Skew ---------------------------------- #

In [ ]:
def calReturnSkew(period):
    AdjPrice = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0)
    returnDF = AdjPrice.pct_change()
    newdf = returnDF.rolling(min_periods=120,window=period,center=False).skew()
    newdf.index.name = 'Own_Factor_Return_Skew_%dD' % period
    newdf.to_csv(path+newdf.index.name+'.csv',na_rep='NaN',date_format='%Y%m%d')
    return newdf

In [ ]:
calReturnSkew(250).tail()

In [ ]:
# --------------------------------------- Calculate  x-days return above 20 days MA  ---------------------------------- #

In [ ]:
def calAbove20MA(period):
    AdjPrice = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0)
    newdf = AdjPrice - AdjPrice.rolling(min_periods=20,window=period,center=False).mean()
    newdf = newdf.rolling(min_periods=20,window=period,center=False).mean()
    newdf.index.name = 'Own_Factor_Above20MA_%dd' % period
    newdf.to_csv(path+'Own_Factor_Above20MA_%dd.csv' % period,na_rep='NaN',date_format='%Y%m%d')
    return newdf 

In [ ]:
calAbove20MA(20).tail()

In [ ]:
# --------------------------------------- Calculate  ARoon  ---------------------------------- #

In [ ]:
AdjPrice = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0).iloc[-1000:]
adcopy = AdjPrice.copy()

In [ ]:
adcopy.iloc[:,2].head()

In [ ]:
def calAroon(data, l=20):
    datacopy = data.copy()
    for i in range(l,len(datacopy)):
        s = datacopy.iloc[i-l:i]
        #print s
        try:
            data.iloc[i] = pd.Timedelta(s.idxmax().date() - s.idxmin().date()).days/l
        except:
            data.iloc[i] = np.NaN
    return data

In [ ]:
#aroonData = adcopy.iloc[:,:10].apply(calAroon,l=20)

In [ ]:
# --------------------------------------- Aroon End  ---------------------------------- #

In [ ]:
# --------------------------------------- Calculate  daily deal Amount(yuan)   ---------------------------------- #

In [ ]:
filenameTVolume = 'LZ_GPA_QUOTE_TVOLUME.csv' # 成交量
filenameAdjPrice =  'Own_Factor_AdjustedPriceForward-1d.csv'

In [ ]:
def calcDDA():
    turnOver = pd.read_csv(path+filenameTVolume,infer_datetime_format=True,parse_dates=[0],index_col=0)
    closePrice = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0)
    #volume = pd.read_csv(path+filenameVolume,infer_datetime_format=True,parse_dates=[0],index_col=0)
    if turnOver.shape != closePrice.shape:
        print turnOver.shape, closePrice.shape
        print 'data shape is not equal!'
    else:
        newdf = turnOver * closePrice
        #newdf = newdf.rolling(min_periods=5,window=5,center=False).mean()
        newdf.index.name = 'Own_Factor_DDA-1d'
        newdf.to_csv(path+'Own_Factor_DDA-1d.csv',na_rep='NaN',date_format='%Y%m%d')
        return newdf           

In [ ]:
calcDDA()

In [ ]:
filenameDDA = 'Own_Factor_DDA-1d.csv'
DDAdf = pd.read_csv(path+filenameDDA,infer_datetime_format=True,parse_dates=[0],index_col=0)

In [ ]:
def DDAaverage(period):
    DDAmean = DDAdf.rolling(min_periods=20,window=period,center=False).mean()
    DDAmean.index.name = 'Own_Factor_DDA-%dd' % period
    DDAmean.to_csv(path+'Own_Factor_DDA-%dd.csv' % period, na_rep='NaN',date_format='%Y%m%d')
    return DDAmean

In [ ]:
DDA20df= DDAaverage(20)

In [ ]:
# TO nuetralize DDA20
NormalizedDDA20 = simpleNormalize(DDA20df)
neutralizedDDA20 = neutralizeFactor(NormalizedDDA20, NormalizedFCAP, NormalizedDDA20.index)
neutralizedDDA20.index.name = 'Own_Factor_ADJ_DDA_20D'
neutralizedDDA20.to_csv(path+neutralizedDDA20.index.name+'.csv',na_rep='NaN',date_format='%Y%m%d')

In [ ]:
# --------------------------------------- Calculate  annual idiosyncratic volatility(daily updated)   ---------------------------------- #

In [ ]:
def cal_factor_return(factor, stkreturn, factorname):
    '''
    To cal the return of the factor by group method( best(1/3) - worst(1/3))
    Output: Dataframe, the factor Return with only one value column
    Input:
    factor: Dataframe, the factor Value df, Note that shift(1) has been used before this function is used
    stkreturn:  Dataframe, the return of all stock in the market. No shift needed.
    factorname:  String, the name of the factor!
    '''
    factorReturn = pd.DataFrame(index=factor.index[1:-1], columns=[factorname], data =None , dtype =float)
    for date in factorReturn.index:
        factorSlice = factor.loc[date].dropna()
        stkreturnSlice = stkreturn.loc[date].dropna()
        intersection = list(set(factorSlice.index) & set(stkreturnSlice.index))
        factorSlice = factorSlice.loc[intersection]
        stkreturnSlice = stkreturnSlice.loc[intersection]
        
        q_min = factorSlice.quantile(0.33)
        q_max = factorSlice.quantile(0.66)
        q_min_univ = factorSlice[factorSlice<=q_min]
        q_max_univ = factorSlice[factorSlice>=q_max]
        q_min_return = (q_min_univ * stkreturnSlice.loc[q_min_univ.index]).sum() / q_min_univ.sum()
        q_max_return = (q_max_univ * stkreturnSlice.loc[q_max_univ.index]).sum() / q_max_univ.sum()
        #print q_min_return
        factorReturn.loc[date] = q_min_return - q_max_return
    return factorReturn

In [ ]:
startTime =  datetime.strptime('20100101', '%Y%m%d')
endTime = datetime.strptime('20170328', '%Y%m%d')
filenameAdjPrice =  'Own_Factor_AdjustedPriceForward-1d.csv'
filenameFCAP = 'LZ_GPA_VAL_A_FCAP.csv'
filenamePB='LZ_GPA_VAL_PB.csv'  # 市净率
filenameBENCH = 'LZ_GPA_INDXQUOTE_CLOSE.csv'
ZZ500Index = '000905.SH' #   ZZ500 index code

In [ ]:
pricedf = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0).loc[startTime:endTime]
FCAPdf  = pd.read_csv(path+filenameFCAP,infer_datetime_format=True,parse_dates=[0],index_col=0).loc[startTime:endTime]
PBdf = pd.read_csv(path+filenamePB,infer_datetime_format=True,parse_dates=[0],index_col=0).loc[startTime:endTime]
benchmarkdf = pd.read_csv(path+filenameBENCH,infer_datetime_format=True,parse_dates=[0],index_col=0)[ZZ500Index].loc[startTime:endTime].pct_change()
returndf = pricedf.pct_change()
FCAPdf = FCAPdf .shift(1)
PBdf = PBdf.shift(1)
FCAPdf.tail()

In [ ]:
SizeReturn = cal_factor_return(FCAPdf, returndf , 'Size_Return')

In [ ]:
SizeReturn.isnull().sum()

In [ ]:
PBReturn = cal_factor_return(PBdf, returndf , 'PB_Return')

In [ ]:
PBReturn.isnull().sum()

In [ ]:
benchmarkdf = benchmarkdf.loc[PBReturn.index[0]:PBReturn.index[-1]]

In [ ]:
benchmarkdf .isnull().sum()

In [ ]:
returndf = returndf.loc[PBReturn.index[0]:PBReturn.index[-1]]

In [ ]:
startOfMonthList, endOfMonthList = ff.getLastDayOfMonth(PBReturn.index)

In [ ]:
len(startOfMonthList)

In [ ]:
len(endOfMonthList)

In [ ]:
datetuple = list(zip(startOfMonthList,endOfMonthList))

In [ ]:
def calIdoVol(returnDF , factorReturnList , datetuple ,torelance = 0.05):
    '''
    function to calculate to idiosyncratic volatility.
    Output: Dataframe, The idiosyncratic volatility factor (same shape as ohter daily factor).
    Input : 
    returnDF: Dataframe, the data of the return of all stocks.
    factorReturn: List, the element is the factor-return dataframe. (Obtained by group-method so there is only one value \
    column(and one index column)
    for each dataframe).Note that this df has been shift(1) to ensure that future data is not used!
    datatuple: List, element is the tuple which is consisted of startTime and endTime. Usually zip by the startOfMonthList and endOfMonthList.
    torelance: float, to filter the Nan Value.
    '''
    startdf = pd.DataFrame()
    for i in datetuple:        
        returnDFSlice = returnDF.loc[i[0]:i[1]]
        tempdf= pd.DataFrame(index = returnDFSlice.index, columns = returnDFSlice.columns ,data = None ,dtype =float)
        newReturnSlice = returnDFSlice.loc[:,returnDFSlice.isnull().sum() < returnDFSlice.shape[1] * torelance]
        newReturnSlice = newReturnSlice.fillna(method = 'ffill').fillna(method = 'bfill')
        filterList = newReturnSlice.columns.tolist()
        mapfunction = map(lambda x: x.loc[i[0]:i[1]], factorReturnList)
        totaldf = pd.concat(mapfunction, axis=1) 
        for stk in filterList:
            result = sm.OLS(newReturnSlice[stk],totaldf).fit()
            tempdf[stk].loc[i[0]:i[1]] = np.std(result.resid) * np.sqrt(242)
        startdf = startdf.append(tempdf)
    return startdf        

In [ ]:
dfList = [PBReturn,SizeReturn,benchmarkdf]

In [ ]:
dd = calIdoVol(returndf , dfList,datetuple)

In [ ]:
dd.index.name = 'Own_FACTOR_Idiosyncratic_Volatility'
dd.head()

In [ ]:
dd.to_csv(path+'Own_Factor_Idiosyncratic_Volatility.csv', na_rep='NaN',date_format='%Y%m%d')

In [ ]:
# --------------------------------------- Calculate  annual idiosyncratic volatility(daily updated)   ---------------------------------- #

In [ ]:
ddc =dd.tail(100)

In [ ]:
filenameIDIVOL = 'Own_Factor_Idiosyncratic_Volatility.csv'
idio = pd.read_csv(path+filenameIDIVOL,infer_datetime_format=True,parse_dates=[0],index_col=0)

In [ ]:
idio.head()

In [ ]:
dd.head()

In [ ]:
def adj_boxplot(factorData):
    '''To calculate  adjusted -boxplot winsorized data and then Normalize the outcome
    Output: Dataframe, the winsorized and normalized data
    Input: 
    factorData:Dataframe, raw data, can contain nan value
    '''
    copyData = factorData.copy()
    for i in copyData.index:
        temp = copyData.loc[i]
        x = temp.dropna().values
        if len(x) > 0:
            mc = sm.stats.stattools.medcouple(x)
            x.sort()
            q1 = x[int(0.25*len(x))]
            q3 = x[int(0.75*len(x))]
            iqr = q3-q1
            if mc >= 0:
                l = q1-1.5*np.exp(-3.5*mc)*iqr
                u = q3+1.5*np.exp(4*mc)*iqr
            else:
                l = q1-1.5*np.exp(-4*mc)*iqr
                u = q3+1.5*np.exp(3.5*mc)*iqr
            temp.loc[temp < l] = l
            temp.loc[temp > u] = u
            #factor_data.loc[i] = (temp-temp.mean())/temp.std()
    Trans = copyData.T
    return ((Trans  - Trans .mean(axis=0, skipna=True))/Trans .std(axis=0, skipna=True)).T

In [ ]:
%mprun  addd = adj_boxplot(dd)

In [ ]:
%lprun -f adj_boxplot adj_boxplot(idio)

In [ ]:
# --------------------------------------- Calculate  some random factor  ---------------------------------- #

In [ ]:
filenameAdjPrice =  'Own_Factor_AdjustedPriceForward-1d.csv'
filenameVolume = 'LZ_GPA_QUOTE_TVOLUME.csv'
closePrice = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0)
tradVol = pd.read_csv(path+filenameVolume,infer_datetime_format=True,parse_dates=[0],index_col=0)

In [ ]:
def calrandfac(pricedf, factordf):
    sgndf= np.sign(pricedf.pct_change())
    tempo = factordf * sgndf
    tempo = tempo.ewm(ignore_na=True, min_periods=5, halflife = 5).mean()
    return tempo

In [ ]:
overheatVol = calrandfac(closePrice, tradVol)

In [ ]:
overheatVol

In [ ]:
overheatVol.index.name = 'OVER_HEAT_VOL'
overheatVol.to_csv(path+'Over_Heat_Volume.csv', na_rep='NaN',date_format='%Y%m%d')

In [ ]:
# --------------------------------------- Calculate  skewness  ---------------------------------- #
filenameAdjPrice =  'Own_Factor_AdjustedPriceForward-1d.csv'
closePrice = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0)

In [ ]:
# calculate N-days skewness of the price
def calSkewness(pricedf,period):
    df = pricedf.rolling(min_periods=250,window=period,center=False).skew()
    df.index.name =  'Own_Factor_Skewness_%dd' % period
    df.to_csv(path+'Own_Factor_Skewness_%dd.csv' % period,na_rep='NaN',date_format='%Y%m%d')
    return df

In [ ]:
priceSkewness = calSkewness(closePrice,250)

In [ ]:
# --------------------------------------- Calculate  TurnOver Rate Volatility ---------------------------------- #
filenameTOR= 'LZ_GPA_VAL_TURN.csv'
turnoverdf =  pd.read_csv(path+filenameTOR,infer_datetime_format=True,parse_dates=[0],index_col=0)
FCAP1 = np.log10(pd.read_csv(path+filenameFCAP,infer_datetime_format=True,parse_dates=[0],index_col=0))

In [ ]:
def calToRvol(period):
    newdf = turnoverdf.rolling(min_periods=20,window=period,center=False).std()
    newdf.index.name = 'Own_Factor_Turnover_Volatility_%dD' % period
    newdf.to_csv(path+'Own_Factor_Turnover_Volatility_%dD.csv' % period,na_rep='NaN',date_format='%Y%m%d')
    return newdf

In [ ]:
newsdf = calToRvol(20)

In [ ]:
newsdf.tail()

In [ ]:
NormalizedTOV = simpleNormalize(newsdf )
NormalizedFCAP = simpleNormalize(FCAP1 )

In [ ]:
neutralizedTOV = neutralizeFactor(NormalizedTOV, NormalizedFCAP, NormalizedTOV.index)

In [ ]:
neutralizedTOV.index.name = 'Own_Factor_ADJ_Turnover_Volatility_20D'
neutralizedTOV.to_csv(path+neutralizedTOV.index.name+'.csv',na_rep='NaN',date_format='%Y%m%d')

In [ ]:
def calToRVolD(period):
    newdf = turnoverdf.rolling(min_periods=20,window=period,center=False).mean()
    newdf1 = turnoverdf.rolling(min_periods=500,window=500,center=False).mean()
    newdf = newdf / newdf1 -1
    newdf.index.name = 'Own_Factor_Turnover_Volatility_deviation_%dD' % period
    newdf.to_csv(path+'Own_Factor_Turnover_Volatility_deviation_%dD.csv' % period,na_rep='NaN',date_format='%Y%m%d')
    return newdf

In [ ]:
df2 = calToRVolD(20)

In [ ]:
df2 .tail()

In [ ]:
NormalizedTOVD = simpleNormalize(df2)
neutralizedTOVD = neutralizeFactor(NormalizedTOVD, NormalizedFCAP, NormalizedTOVD.index)
neutralizedTOVD.index.name = 'Own_Factor_ADJ_Turnover_Volatility_Deviation_20D'
neutralizedTOVD.to_csv(path+neutralizedTOVD.index.name+'.csv',na_rep='NaN',date_format='%Y%m%d')

In [ ]:
neutralizedTOVD .tail()

In [ ]:
# --------------------------------------- Calculate  adjusted Own factor  ILLQ  ---------------------------------- #